<a href="https://colab.research.google.com/github/Amey-Thakur/ZERO-SHOT-VIDEO-GENERATION/blob/main/Source%20Code/Zero_Shot_Video_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#
<h1 align="center">🎬 Zero-Shot Video Generation</h1>
<h3 align="center"><i>Text-to-Video Synthesis via Temporal Latent Warping & Cross-Frame Attention</i></h3>

<div align="center">

| **Author** | **Profiles** |
|:---:|:---|
| **Amey Thakur** | [![GitHub](https://img.shields.io/badge/GitHub-Amey--Thakur-181717?logo=github)](https://github.com/Amey-Thakur) [![ORCID](https://img.shields.io/badge/ORCID-0000--0001--5644--1575-A6CE39?logo=orcid)](https://orcid.org/0000-0001-5644-1575) [![Google Scholar](https://img.shields.io/badge/Google_Scholar-Amey_Thakur-4285F4?logo=google-scholar&logoColor=white)](https://scholar.google.ca/citations?user=0inooPgAAAAJ&hl=en) [![Kaggle](https://img.shields.io/badge/Kaggle-Amey_Thakur-20BEFF?logo=kaggle)](https://www.kaggle.com/ameythakur20) |

---

**Research Foundation:** Based on [Text2Video-Zero](https://arxiv.org/abs/2303.13439) by the Picsart AI Research (PAIR) team.

🚀 **Live Demo:** [Hugging Face Space](https://huggingface.co/spaces/AmeyThakur/ZERO-SHOT-VIDEO-GENERATION) | 🎬 **Video Demo:** [YouTube](https://youtu.be/za9hId6UPoY) | 💻 **Repository:** [GitHub](https://github.com/Amey-Thakur/ZERO-SHOT-VIDEO-GENERATION)

</div>

## 📖 Introduction

> **An audio-visual deepfake is when temporally consistent synthetic content is generated without requiring person-specific training.**

This research implementation demonstrates **Text2Video-Zero**, a state-of-the-art framework that transforms pre-trained Text-to-Image (T2I) diffusion models into zero-shot video generators. By enforcing temporal consistency through latent manipulation and attention synchronization, we can synthesize high-quality videos without any video-specific fine-tuning.

### Core Innovations:
1.  **Temporal Latent Warping**: Enforces geometric consistency by warping latents along a motion field.
2.  **Global Cross-Frame Attention**: Synchronizes object identity by making all frames attend to the appearance of the first frame.
3.  **Background Smoothing**: Separates foreground dynamics from background stability to reduce pixel flickering.

## ☁️ Cloud Environment Setup
Execute the following cell to configure your environment. This script is **fail-proof** and **multi-tier fallback** optimized.

### Operational Workflow:
1.  **Environment Detection**: Automatically detects **Kaggle**, **Colab**, or **Local** execution.
2.  **Robust Cloning**: Attempts GitHub first, with a fallback to **Hugging Face Spaces** if LFS budget is exceeded.
3.  **Asset Synchronization**: Automatically downloads missing neural weights (model checkpoints and annotators) from the Hugging Face Hub if they aren't provided in the local filesystem.

In [ ]:
import os
import sys
import shutil
import subprocess

# ── Detect Environment ───────────────────────────────────────────────────────
try:
    shell = get_ipython()
    IS_COLAB = 'google.colab' in str(shell)
    IS_KAGGLE = "kaggle" in os.environ.get("KAGGLE_KERNEL_RUN_TYPE", "")
except NameError:
    IS_COLAB = IS_KAGGLE = False

PROJECT_NAME = "ZERO-SHOT-VIDEO-GENERATION"
print(f"🌍 Environment: {'Google Colab' if IS_COLAB else ('Kaggle' if IS_KAGGLE else 'Local/Custom')}")

def run_setup():
    if IS_COLAB or IS_KAGGLE:
        WORKDIR = "/content" if IS_COLAB else "/kaggle/working"
        os.chdir(WORKDIR)
        
        # 1. Clone Repository (with Fallback)
        if not os.path.exists(PROJECT_NAME):
            print(f"⬇️ Cloning {PROJECT_NAME} from GitHub...")
            res = os.system(f"git clone https://github.com/Amey-Thakur/{PROJECT_NAME}")
            
            if res != 0 or not os.path.exists(PROJECT_NAME):
                print("⚠️ GitHub Clone Failed or LFS Error. Falling back to Hugging Face Mirror...")
                os.system(f"git clone https://huggingface.co/spaces/AmeyThakur/{PROJECT_NAME}")
        
        os.chdir(os.path.join(WORKDIR, PROJECT_NAME, "Source Code"))
        
        # 2. Dependency Installation
        print("🛠️ Installing Dependencies...")
        os.system("pip install -q diffusers transformers accelerate einops kornia imageio imageio-ffmpeg moviepy tomesd decord safetensors huggingface_hub ipywidgets")
        
        # 3. Comprehensive Asset Synchronization
        print("📦 Synchronizing neural weights...")
        from huggingface_hub import hf_hub_download
        
        # Annotator Checkpoints
        annotators = {
            "body_pose_model.pth": "lllyasviel/Annotators",
            "hand_pose_model.pth": "lllyasviel/Annotators",
            "dpt_hybrid-midas-501f0c75.pt": "lllyasviel/Annotators",
            "upernet_global_small.pth": "lllyasviel/Annotators"
        }
        
        os.makedirs("annotator/ckpts", exist_ok=True)
        for filename, repo_id in annotators.items():
            target = os.path.join("annotator/ckpts", filename)
            if not os.path.exists(target) or os.path.getsize(target) < 1000:
                print(f"📥 Retrieving {filename}...")
                try:
                    path = hf_hub_download(repo_id=repo_id, filename=filename)
                    shutil.copy(path, target)
                except Exception as e:
                    print(f"⚠️ Download Failed: {e}")
                
    print("✅ Environment Setup Complete.")

run_setup()

# Add Source Code to path for module imports
current_path = os.getcwd()
if current_path not in sys.path:
    sys.path.append(current_path)
print(f"📍 Workspace: {current_path}")

## 1️⃣ Pipeline & HW Initialization
We initialize the Text2Video architecture. A **GPU with at least 15GB VRAM** is recommended for high-resolution output.

In [ ]:
import torch
import gc
from diffusers import DDIMScheduler
from model import Model, ModelType

# ── Hardware Diagnostics ─────────────────────────────────────────────────────
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16 if device == "cuda" else torch.float32
print(f"🎯 Computation Device: {device}")
print(f"💎 Precision Mode:    {dtype}")

if device == "cuda":
    print(f"📟 GPU:               {torch.cuda.get_device_name(0)}")
    vram = torch.cuda.get_device_properties(0).total_mem / 1024**3
    print(f"📊 Total VRAM:        {vram:.2f} GB")

# ── Architecture Initialization ──────────────────────────────────────────────
print("⏳ Loading Neural Networks (Zero-Shot Pipeline)...")
model = Model(device=device, dtype=dtype)

def cleanup_vram():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

print("✅ Pipeline Operational.")

## 2️⃣ Synthesis Studio
Configure your parameters below. This interactive studio allows you to fine-tune the motion dynamics and temporal consistency of the generated video.

In [ ]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import numpy as np
import base64

# ── Presets ──────────────────────────────────────────────────────────────────
EXAMPLES = [
    "an astronaut waving the arm on the moon",
    "a sloth surfing on a wakeboard",
    "a cute cat walking on grass",
    "a horse is galloping on a street",
    "a gorilla dancing on times square"
]

# ── UI Components ────────────────────────────────────────────────────────────
header = widgets.HTML("<h3>🎬 Video Generation Controls</h3>")

preset_dropdown = widgets.Dropdown(
    options=[("Select a preset...", "")] + [(p, p) for p in EXAMPLES],
    description='<b>Presets:</b>',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px')
)

prompt_input = widgets.Textarea(
    value='an astronaut waving the arm on the moon',
    placeholder='Type your creative prompt here...',
    description='<b>Prompt:</b>',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='500px', height='80px')
)

length_slider = widgets.IntSlider(value=8, min=4, max=24, step=1, description='Video Length (Frames)')
res_dropdown = widgets.Dropdown(options=[256, 512, 768], value=512, description='Resolution')
motion_slider = widgets.FloatSlider(value=12.0, min=0.0, max=30.0, step=0.5, description='Motion Strength')
steps_slider = widgets.IntSlider(value=50, min=10, max=100, description='Inference Steps')
fps_slider = widgets.IntSlider(value=4, min=1, max=12, description='Playback FPS')
seed_input = widgets.IntText(value=42, description='Seed')

process_btn = widgets.Button(
    description='🚀 Generate Video',
    button_style='primary',
    layout=widgets.Layout(width='500px', height='40px')
)

output_vram = widgets.Output()
output_video = widgets.Output()

# ── Logic ────────────────────────────────────────────────────────────────────
def on_preset_change(change):
    if change['new']:
        prompt_input.value = change['new']

preset_dropdown.observe(on_preset_change, names='value')

def run_generation(b):
    process_btn.disabled = True
    process_btn.description = "⏳ Processing... Please wait..."
    
    with output_video:
        clear_output()
        print("\n🚀 Initiating generation sequence...")
        
        output_path = "output.mp4"
        try:
            video_path = model.process_text2video(
                prompt=prompt_input.value,
                video_length=length_slider.value,
                resolution=res_dropdown.value,
                motion_field_strength_x=motion_slider.value,
                motion_field_strength_y=motion_slider.value,
                seed=seed_input.value,
                fps=fps_slider.value,
                path=output_path
            )
            
            # Display Video
            with open(video_path, "rb") as f:
                data = f.read()
            b64 = base64.b64encode(data).decode()
            html = f'''
            <div align="center">
                <br>
                <h4>✨ Generation Complete!</h4>
                <video width="{res_dropdown.value}" controls autoplay loop>
                    <source src="data:video/mp4;base64,{b64}" type="video/mp4">
                </video>
            </div>
            '''
            display(HTML(html))
            
        except Exception as e:
            print(f"❌ Error during synthesis: {e}")
        
        process_btn.disabled = False
        process_btn.description = "🚀 Generate Video"
        cleanup_vram()

process_btn.on_click(run_generation)

# ── Layout ───────────────────────────────────────────────────────────────────
ui = widgets.VBox([
    header,
    preset_dropdown,
    prompt_input,
    widgets.HBox([length_slider, res_dropdown]),
    widgets.HBox([motion_slider, steps_slider]),
    widgets.HBox([fps_slider, seed_input]),
    process_btn,
    output_vram,
    output_video
])

display(ui)

## 📚 References
1. **Text2Video-Zero**: Khachatryan, L. et al. (2023). *Text2Video-Zero: Text-to-Image Diffusion Models are Zero-Shot Video Generators.* [arXiv:2303.13439](https://arxiv.org/abs/2303.13439)
2. **Dreamlike Photoreal 2.0**: [HuggingFace](https://huggingface.co/dreamlike-art/dreamlike-photoreal-2.0)
3. **Diffusers Framework**: [GitHub](https://github.com/huggingface/diffusers)

---

*Research Project | University of Windsor | Authors: Amey Thakur*